In [23]:
import pandas as pd
from rapidfuzz import process, fuzz

In [24]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import logging as transformers_logging
transformers_logging.set_verbosity_error()

class NERParser:
    def __init__(self, model_name: str = "dslim/bert-base-NER", lowercase: bool = False):
        """
        Initialize the NER parser with a model and optionally configure the lowercase preprocessing.
        """
        self.model_name = model_name
        self.lowercase = lowercase
        self.device = self.get_device()
        
        # Load the tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, do_lower_case=self.lowercase)
        self.model = AutoModelForTokenClassification.from_pretrained(self.model_name)
        
        # Set up the NER pipeline
        self.nlp_pipeline = pipeline("ner", 
                                     model=self.model, 
                                     tokenizer=self.tokenizer, 
                                     device=self.device, 
                                     aggregation_strategy="simple")

    def get_device(self):
        """
        Determines whether to use MPS, CUDA, or CPU depending on the available hardware.
        """
        if torch.backends.mps.is_available():
            print("MPS device found, using MPS backend.\n")
            return torch.device("mps")
        elif torch.cuda.is_available():
            print(f"CUDA device found, using CUDA backend. Device: {torch.cuda.get_device_name(0)}\n")
            return torch.device("cuda")
        else:
            print("Neither MPS nor CUDA found, using CPU.\n")
            return torch.device("cpu")

    
    def parse_ner_results(self, ner_results: list):
        """
        Parse the NER results and extract entities related to 'PER' (persons) and 'MISC' (potential movie titles).
        """
        per_entities, misc_entities = [], []
        
        for entity in ner_results:
            # Extraction of all Persons
            if entity['entity_group'] == 'PER':
                per_entities.append(entity['word'])
            # Extraction of all Misc that could indicate movies
            elif entity['entity_group'] == 'MISC':
                misc_entities.append(entity['word'])
        
        return per_entities, misc_entities

    
    def process_query(self, query: str):
        """
        Processes a text query, runs NER, and returns the extracted actors and movie names.
        """
        # Optionally lowercase the input if configured
        if self.lowercase:
            query = query.lower()
        
        # Run the NER pipeline
        ner_results = self.nlp_pipeline(query)

        # Parse the results to extract actors and movies
        per_entities, misc_entities = self.parse_ner_results(ner_results)
        
        return per_entities, misc_entities

ner_parser = NERParser(lowercase=False)

Neither MPS nor CUDA found, using CPU.



In [25]:
db = pd.read_pickle('exports/graph.pkl')

In [26]:
query = "Where was Angelina Jolie born?"
person, movies = ner_parser.process_query(query)

In [27]:
context_person = []

for p in person:
    # Exact Lookup
    relevant = db[db.index == p].dropna(axis=1)
    
    if not relevant.empty:
        context_person.append(relevant.iloc[0])
    
    else:
        # Levenshtein matching for closest match
        matches = process.extractOne(p, db.index, scorer=fuzz.ratio)
        
        if matches and matches[1] > 80:
            closest_match = matches[0]
            relevant = db[db.index == closest_match].dropna(axis=1)
            context_person.append(relevant.iloc[0])
   
context_df_person = pd.DataFrame(context_person)

context_movie = [] 
   
for m in movies:
    # Exact Lookup
    relevant = db[db.index == m].dropna(axis=1)
    
    if not relevant.empty:
        context_movie.append(relevant.iloc[0])
    
    else:
        # Levenshtein matching for closest match
        matches = process.extractOne(m, db.index, scorer=fuzz.ratio)
        
        if matches and matches[1] > 80:
            closest_match = matches[0]
            relevant = db[db.index == closest_match].dropna(axis=1)
            context_movie.append(relevant.iloc[0])   
        
context_df_movie = pd.DataFrame(context_movie)

In [28]:
context_df_movie

""


In [29]:
context_df_person.columns

Index(['IMDb ID', 'ancestral home', 'award received', 'country of citizenship',
       'described by source', 'father', 'image', 'instance of',
       'languages spoken, written or signed', 'mother', 'native language',
       'node description', 'node label', 'nominated for', 'notable work',
       'occupation', 'place of birth', 'relative', 'religion', 'residence',
       'sibling', 'significant event', 'spouse', 'unmarried partner'],
      dtype='object', name='Predicate')

In [33]:
import requests

url = 'http://nlp.stanford.edu/data/glove.6B.zip'
response = requests.get(url)

with open('input/glove.6B.zip', 'wb') as file:
    file.write(response.content)

In [37]:
import zipfile
import os

# Assuming the zip file is in the current directory
zip_file_path = 'input/glove.6B.zip'
extract_to_path = 'exports/glove.6B'  # Extracting to the current directory

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Unzipping complete!")


Unzipping complete!


In [38]:
import numpy as np

def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Load the GloVe embeddings (300-dimensional)
glove_embeddings = load_glove_embeddings("exports/glove.6B/glove.6B.300d.txt")

In [39]:
def embed_phrase(phrase, embeddings):
    words = phrase.split()
    word_vectors = [embeddings[word.lower()] for word in words if word.lower() in embeddings]
    
    if len(word_vectors) == 0:
        return np.zeros(300)  # Return zero vector if no words have embeddings
    
    return np.mean(word_vectors, axis=0)

# Embed column names of knowledge graph db
column_embeddings = {col: embed_phrase(col, glove_embeddings) for col in context_df_person.columns}

In [40]:
column_embeddings

{'IMDb ID': array([ 0.24651998,  0.07036   , -0.14642501,  0.116624  ,  0.056493  ,
        -0.22114   , -0.09043   , -0.075065  ,  0.00623   ,  0.04271001,
         0.137815  ,  0.33861   ,  0.440625  ,  0.16979034, -0.35968   ,
         0.061608  , -0.174365  , -0.03683   , -0.22728   , -0.42122   ,
         0.03673501,  0.04362   , -0.528735  , -0.296985  ,  0.07916079,
         0.24066399,  0.120575  ,  0.462405  ,  0.18459   , -0.377335  ,
        -0.31038502, -0.0232805 , -0.04938   ,  0.31717497,  0.005455  ,
        -0.235765  , -0.265915  , -0.0691155 , -0.132442  , -0.470625  ,
         0.28242052, -0.412005  ,  0.32108   ,  0.62975   , -0.30144352,
        -0.18836328,  0.036367  , -0.484735  , -0.39131   , -0.63302004,
         0.0996885 ,  0.139675  ,  0.49345   , -0.405625  ,  0.026585  ,
        -0.12878   ,  0.1299315 , -0.229645  , -0.50788   , -0.2958    ,
        -0.173519  ,  0.2378495 , -0.11698   ,  0.134761  , -0.13264701,
        -0.10877001, -0.159163  ,  0.019

In [42]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [43]:
def filter_query(query):
    
    person_list = [p.split(" ") for p in person][0] if len(person) else []
    movie_list = [m.split(" ") for m in movies][0] if len(movies) else []
        
    if not len(query):
        return []
    
    relevant = []
    for word in query.split(" "):
        if word in stop_words:
            continue
        
        if word in person_list:            
            continue
        
        if word in movie_list:
            continue
        
        relevant.append(word)
        
    return relevant
    

In [44]:
# Embed the query keywords
query = "Where was Angelina Jolie born?"
filtered_query = filter_query(query)
query_embeddings = [embed_phrase(word, glove_embeddings) for word in filtered_query]


In [45]:
def cosine_sim(vec1, vec2):
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0.0 
    
    return np.dot(vec1, vec2) / (norm_vec1 * norm_vec2)

def find_closest_columns(query_embeddings, column_embeddings, top_n=5):
    column_similarities = {}

    for col, col_vec in column_embeddings.items():
        similarities = [cosine_sim(col_vec, q_vec) for q_vec in query_embeddings if np.linalg.norm(q_vec) > 0]
        column_similarities[col] = np.mean(similarities) if similarities else -1

    sorted_columns = sorted(column_similarities.items(), key=lambda item: item[1], reverse=True)
    
    return [col for col, sim in sorted_columns[:top_n]]

top_columns = find_closest_columns(query_embeddings, column_embeddings, top_n=5)

print("Top columns:", top_columns)

Top columns: ['instance of', 'place of birth', 'country of citizenship', 'ancestral home', 'notable work']


In [46]:
context_df = pd.concat([context_df_person, context_df_movie], axis=0)

In [47]:
context_df[top_columns]

Predicate,instance of,place of birth,country of citizenship,ancestral home,notable work
Angelina Jolie,human,Los Angeles,"United States of America, Cambodia",Germany,"Girl, Interrupted, Gia, Changeling"


In [48]:
# Introduce a swift llm that can generate the answer based on the context_df
from transformers import pipeline

# Load pre-trained DistilBERT model for question answering
qa_model = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

C:\Users\sandr\UntrackedFolder\movie-bot\venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sandr\.cache\huggingface\hub\models--distilbert-base-cased-distilled-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [49]:
# Context and query
context = ""
for index, row in context_df.iterrows():
    row_context = " ".join([f"{col}: {row[col]}" for col in context_df[top_columns].columns])
    context += row_context + " "
    
query = "Where was Angelina Jolie born?"

# Generate the answer
result = qa_model(question=query, context=context)

print(result['answer']) 

Los Angeles
